In [2]:
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(
    temperature=0.2,
    # groq_api_key="add_your_api_key",
    
    model_name="llama-3.3-70b-versatile",
)

In [5]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-36947?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for Product Developer II, Global Apparel

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsProduct Creation, Development & ManagementProduct Developer II, Global ApparelHong Kong Is

In [6]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """ 
    ### SCRAPED TEXT FROM WEBSITE :
    {page_data}
    ### INSTRUCTIONS:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the 
    following keys: `role`, `experience`, `skills` and `description`. 
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input = {'page_data': page_data})
print(res.content)

```json
{
  "role": "Product Developer II, Global Apparel",
  "experience": "At least 3 years of relevant experience",
  "skills": [
    "Apparel development and manufacturing",
    "Knowledge of apparel manufacturing, costing and garment fit",
    "Project management and problem solving skills",
    "Team player with ability to work in an environment with high inter-dependencies between teams and cross-cultural colleagues",
    "Proficient with Microsoft Excel, PowerPoint, Word, and Outlook"
  ],
  "description": "The Product Developer coordinates the product creation process from initial concept to commercialization and production for assigned models. Leads on time delivery and development according to critical dates, product specification, sustainability, performance, and profitability goals. Collaborates with Design, Brand and Category Management as well as Merchandising, and Manufacturing to effectively execute product development goals and deliverables."
}
```
